## Purpose

This notebook identifies potential duplicate records within your data using vector similarity search.

## Prerequisites

*   A Databricks workspace with access to Databricks Vector Search.
*   A Model Serving endpoint serving an embedding model (e.g., `databricks-gte-large-en`).
*   Bronze tables with the data you want to analyze, with appropriately defined `primaryKey`

## Outputs

*   Creates temporary views named `<entity_name>_duplicate_candidates` for each entity in the provided data model. These views contain potential duplicate records and their similarity scores.

This is your first step in identifying and deduplicating your data!  Run this first to create the datasets that tell you your duplicate candidates

In [0]:
%run ./config/include/multiThreading

In [0]:
dbutils.widgets.text("embedding_model_endpoint_name", "databricks-gte-large-en", "Embedding Model")
dbutils.widgets.text("num_results", "3", "Num Results")
dbutils.widgets.text("catalog_name","")
dbutils.widgets.text("vector_search","lakefusion_vs_endpoint","Vector Search Name")

In [0]:
embedding_model_endpoint_name = dbutils.widgets.get("embedding_model_endpoint_name")
num_results = int(dbutils.widgets.get("num_results"))
catalog_name = dbutils.widgets.get("catalog_name")
vector_search = dbutils.widgets.get("vector_search")
if not catalog_name:
    raise Exception("Catalog name is required to run this notebook")

In [0]:
notebookPath = current_notebook_path.replace(current_notebook_path.split("/")[-1],"config/detect_duplicates_main")

In [0]:
# Example usage:  Hardcoded Entities
entities = [
    {"table_name": f"{catalog_name}.bronze.payer", "primary_key": "payer_id", "columns_to_exclude": ["payer_id"]},
    {"table_name": f"{catalog_name}.bronze.payer_accreditation", "primary_key": "accreditation_id", "columns_to_exclude": ["accreditation_id", "payer_id"]},
    {"table_name": f"{catalog_name}.bronze.payer_address", "primary_key": "address_location_id", "columns_to_exclude": ["address_location_id", "patient_id"]},
    {"table_name": f"{catalog_name}.bronze.payer_alt_identifiers", "primary_key": "identifier_id", "columns_to_exclude": ["identifier_id", "patient_id"]},
    {"table_name": f"{catalog_name}.bronze.payer_alt_name", "primary_key": "alt_name_id", "columns_to_exclude": ["alt_name_id", "patient_id"]},
    {"table_name": f"{catalog_name}.bronze.payer_contact_information", "primary_key": "contact_id", "columns_to_exclude": ["contact_id", "patient_id"]},
    {"table_name": f"{catalog_name}.bronze.payer_network", "primary_key": "payer_network_id", "columns_to_exclude": ["payer_network_id", "patient_id"]}
    ]

In [0]:
notebooks = [NotebookData(f"{notebookPath}", 3600, {"table_name" : f'{entity["table_name"]}',"columns_to_exclude" : f'{entity["columns_to_exclude"]}',"primary_key" : f'{entity["primary_key"]}',"catalog_name" : f'{catalog_name}',"num_results" : f'{num_results}',"vector_search" : f'{vector_search}',"embedding_model_endpoint_name" : f'{embedding_model_endpoint_name}'}) for entity in entities]   

# #Array of instances of NotebookData Class
parallel_thread = 4

try : 
    res = parallel_notebooks(notebooks, parallel_thread)
    result = [i.result(timeout = 3600) for i in res] # This is a blocking call.
    print(result)  
except NameError as e :
    print(e)